# CITS3401 Project 1
### Kean Scott - 23850149

## Introduction

In Australia, motor vehicle accidents make up a significant proportion of potentially preventable 

The following python notebook contains all the steps for the analysis of the Australian Road Deaths Database (**ARDD**), also providing the steps taken to build a daata warehouse suitable for ongoing observation of key metrics and statistics. These key metrics will be defined to answer particular questions reguarding road deaths in Australia, in an effort to uncover underlying trends and correlations in these fatalities.  

Here are the original sources of my data:

- [Fatal crashes (updated Feb 2025)](https://catalogue.data.infrastructure.gov.au/dataset/australian-road-deaths-database/resource/457dbf98-419e-4f1e-a45f-4d568ff0ff69?inner_span=True)
- [Fatalities (updated Feb 2025)](https://catalogue.data.infrastructure.gov.au/dataset/australian-road-deaths-database/resource/80091814-9a39-444c-a329-b27561d8fcc6?inner_span=True)

## Step 1: Initial Data Exploration - Identifying the questions to be asked of our data warehouse, and defining it's dimensions

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np 

In [5]:
fatal_crashes = pd.read_csv("data/fatal_crashes_feb2025.csv")
fatalities = pd.read_csv("data/fatalities_feb2025.csv")

In [7]:
fatalities.head()

,Crash ID,State,Month,Year,Dayweek,Time,Crash Type,Bus Involvement,Heavy Rigid Truck Involvement,Articulated Truck Involvement,Speed Limit,Road User,Gender,Age,National Remoteness Areas 2021,SA4 Name 2021,National LGA Name 2021,National Road Type,Christmas Period,Easter Period
0,120161098270,NSW,4,2016,Monday,15:29:00,Multiple,No,No,No,100,Driver,Male,76,Inner Regional Australia,New England and North West,Tamworth Regional,Arterial Road,No,No
1,120161097596,NSW,4,2016,Tuesday,16:40:00,Multiple,No,No,Yes,110,Driver,Female,49,Inner Regional Australia,Far West and Orana,Dubbo Regional,National or State Highway,No,No
2,120161097596,NSW,4,2016,Tuesday,16:40:00,Multiple,No,No,Yes,110,Passenger,Female,80,Inner Regional Australia,Far West and Orana,Dubbo Regional,National or State Highway,No,No
3,120161098282,NSW,4,2016,Sunday,14:00:00,Single,No,No,No,80,Passenger,Male,13,Inner Regional Australia,Riverina,Lockhart,Local Road,No,No
4,120161098913,NSW,4,2016,Saturday,07:30:00,Single,No,No,No,110,Driver,Male,21,Outer Regional Australia,Far West and Orana,Narromine,National or State Highway,No,No


In [8]:
fatal_crashes.head()

,Crash ID,State,Month,Year,Dayweek,Time,Crash Type,Number Fatalities,Bus Involvement,Heavy Rigid Truck Involvement,Articulated Truck Involvement,Speed Limit,National Remoteness Areas 2021,SA4 Name 2021,National LGA Name 2021,National Road Type,Christmas Period,Easter Period
0,620172123,TAS,1,2017,Friday,16:10:00,Multiple,1,No,No,No,60,Inner Regional Australia,Hobart,Hobart,Sub-arterial Road,No,No
1,620172124,TAS,1,2017,Friday,19:00:00,Single,1,No,No,No,100,Outer Regional Australia,Launceston and North East,Northern Midlands,Arterial Road,No,No
2,620172125,TAS,2,2017,Monday,13:50:00,Multiple,1,No,Yes,No,100,Outer Regional Australia,Launceston and North East,Break O'Day,Arterial Road,No,No
3,620172126,TAS,3,2017,Thursday,11:29:00,Single,1,No,No,No,50,Inner Regional Australia,Launceston and North East,West Tamar,Local Road,No,No
4,620172127,TAS,3,2017,Saturday,13:20:00,Multiple,1,No,No,No,100,Remote Australia,West and North West,West Coast,Arterial Road,No,No
